<a href="https://colab.research.google.com/github/weiyuchen1213/CE888/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
jester_df = pd.read_csv("https://raw.githubusercontent.com/weiyuchen1213/CE888/master/Lab5/jester-data-1.csv", delimiter = ",")
jester_df

df_del = jester_df.drop(jester_df.columns[0],axis=1) #Columns "74" 
df_del

In [0]:
arr = df_del.values
print(arr.shape)

In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr

array([[ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  ,  8.35, 99.  , ..., 99.  , 99.  , 99.  ],
       ...,
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [ 2.43,  2.67, -3.98, ..., 99.  , 99.  , 99.  ]])

In [0]:
new_arr

array([[100.  ,   4.08,  -0.29, ...,  99.  ,  99.  ,   1.07],
       [ 99.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 48.  ,  99.  ,   8.35, ...,  99.  ,  99.  ,  99.  ],
       ...,
       [ 39.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 99.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

In [0]:
arr[idx[0][0], idx[1][0]]

1.21

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

array([[ 5.984e+01,  6.071e+01],
       [ 1.848e+00, -3.984e-01],
       [ 1.376e+00, -7.815e-01],
       [ 1.533e+00, -9.180e-01],
       [ 6.233e-01, -2.404e+00],
       [ 1.201e+00, -3.857e-01],
       [ 2.624e+00, -1.492e-01],
       [ 3.454e-01, -7.592e-01],
       [-4.764e-02, -7.198e-01],
       [ 1.172e+00, -1.864e+00],
       [ 2.006e+00, -1.867e-02],
       [ 2.460e+00,  3.994e-01],
       [ 2.238e+00,  1.722e-01],
       [-7.635e-01, -1.837e+00],
       [ 2.370e+00, -2.524e-02],
       [-8.145e-01, -1.754e+00],
       [-1.490e+00, -3.140e+00],
       [-1.748e-01, -1.313e+00],
       [ 2.025e-01, -1.073e+00],
       [ 1.072e+00, -6.213e-01],
       [ 2.538e-01, -1.525e+00],
       [ 2.845e+00,  6.714e-01],
       [ 2.067e+00, -5.625e-01],
       [ 1.487e+00, -1.126e+00],
       [ 5.889e-01, -2.779e+00],
       [ 1.610e+00, -7.746e-01],
       [ 2.168e+00,  3.170e-02],
       [ 3.202e+00,  1.824e+00],
       [ 2.465e+00,  1.819e-02],
       [ 2.987e+00,  1.803e+00],
       [ 1

In [0]:
latent_user_preferences

array([[0.893, 0.709],
       [1.365, 0.985],
       [0.411, 0.416],
       ...,
       [0.311, 0.317],
       [0.246, 0.384],
       [0.972, 0.898]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=10000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if  (rating!=99): #
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)


In [0]:
sgd(1000)

23.317996759523073
18.498982344035728
17.956203689309895
17.904767394132328
17.89182066036718


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 9.665e+01,  1.144e+00,  4.399e-01, ...,  1.017e+00,  1.022e-01,
         1.512e+00],
       [ 1.220e+02,  1.496e+00,  5.841e-01, ...,  1.382e+00,  2.926e-01,
         2.072e+00],
       [ 4.983e+01,  5.816e-01,  2.224e-01, ...,  5.096e-01,  2.765e-02,
         7.548e-01],
       ...,
       [ 3.781e+01,  4.586e-01,  1.783e-01, ...,  4.193e-01,  7.551e-02,
         6.269e-01],
       [ 3.792e+01,  4.273e-01,  1.608e-01, ...,  3.589e-01, -2.740e-02,
         5.263e-01],
       [ 1.012e+02,  1.194e+00,  4.588e-01, ...,  1.059e+00,  9.732e-02,
         1.574e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

ValueError: ignored

In [0]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(100.0, 96.59058309149052)","(4.08, 1.1165225074048182)","(-0.29, 0.3825326104979167)","(6.36, 0.46559920585274056)","(4.37, -1.4992714248220618)","(-2.38, 0.6243523436988867)","(-9.66, 1.95627480759112)","(-0.73, -0.3765304570318841)","(-5.34, -0.6209844629272387)","(8.88, -0.6134710417117507)","(9.22, 1.5810034509131463)","(6.75, 2.2806820954802967)","(8.64, 1.861036274355439)","(99.0, -2.171707956393812)","(7.43, 1.7780433281231633)","(4.56, -2.10074540685998)","(-0.97, -3.783985851669823)","(4.66, -1.2683665927122583)","(-0.68, -0.7353014406868369)","(3.3, 0.30938627520934636)","(-1.21, -1.113379368018086)","(0.87, 2.757749728696722)","(8.64, 1.1215639065917389)","(8.35, 0.2377612391804824)","(9.17, -1.8471920626704932)","(0.05, 0.6079446312065552)","(7.57, 1.6925650161479113)","(4.71, 4.032994813643338)","(0.87, 1.9525121692709342)","(-0.39, 3.7990883214030933)","(99.0, -0.4330972397553946)","(6.5, 2.84949919704973)","(-0.92, 4.0222803112440735)","(7.14, -1.4388814123140097)","(9.03, 1.1703437139875785)","(-1.8, 3.8686325443483858)","(0.73, 4.192938409492768)","(7.09, -1.4896920816862842)","(3.4, 1.6656006380483868)","(-0.87, 1.4756284268747026)",...,"(-6.7, 3.201777329847235)","(-3.35, 3.7982024285500633)","(-9.03, 0.3967543339854653)","(4.47, -0.664124367473831)","(4.08, 2.9818561828225207)","(-3.83, 3.2746124326729498)","(8.74, -0.9141224868264912)","(1.12, 3.353493131899608)","(0.78, 3.351380537235889)","(7.52, 0.6661889780392272)","(-5.0, -0.6323309761542274)","(2.77, 2.932023143462509)","(8.3, 1.2302406209245584)","(7.77, -1.433196794753642)","(7.33, -0.14286033229202597)","(6.21, 2.7013590398344864)","(99.0, 0.8573417677020233)","(8.98, 1.9121088924099623)","(8.64, 0.27175547411668116)","(8.2, 1.2650518027832067)","(99.0, 2.0038695963241024)","(4.85, 1.044836502324691)","(4.85, 2.2662666996945178)","(99.0, 0.8405891371141799)","(8.98, 1.041752483249147)","(4.51, 0.43066668591649626)","(-0.05, 2.0502644328687007)","(3.69, 2.3116959413220877)","(4.56, 3.813825364066786)","(0.58, 0.7697178003624932)","(2.82, 2.2725460328967495)","(-4.95, 1.371920669154244)","(99.0, 2.688232628973198)","(7.86, 1.3575753337749235)","(-0.19, 1.1944768248113775)","(-2.14, 1.7065307679963009)","(3.06, 1.85098761198351)","(99.0, 0.9620992210449821)","(99.0, 0.05227376179324964)","(1.07, 1.4754230527217853)"
1,"(99.0, 98.55500477453235)","(99.0, 1.143267924571677)","(99.0, 0.3949143983096768)","(99.0, 0.4969942803439192)","(99.0, -1.513664143625643)","(9.03, 0.6485408094683769)","(9.27, 2.042483001188604)","(9.03, -0.4260552838445612)","(9.27, -0.6634438660672509)","(99.0, -0.5959103100413748)","(99.0, 1.623023427891907)","(99.0, 2.295722090796993)","(7.57, 1.8636469513772855)","(99.0, -2.2301635480235134)","(99.0, 1.8162404927101854)","(-6.36, -2.1585316877920087)","(99.0, -3.8282800589176404)","(-7.86, -1.2984522871875883)","(9.03, -0.7556812896210393)","(9.03, 0.31788581325099613)","(9.03, -1.1558169211184917)","(7.28, 2.8350503263079188)","(99.0, 1.1487939162439282)","(8.25, 0.23378335894261254)","(99.0, -1.8443105751688271)","(99.0, 0.5828415016744092)","(7.48, 1.686812925371972)","(7.28, 4.091770324722582)","(7.28, 1.986579614970888)","(8.93, 3.8672136853205097)","(99.0, -0.43226681886023954)","(6.17, 2.9298460646343827)","(7.28, 4.087901760922658)","(99.0, -1.4610749736746211)","(99.0, 1.1890744263812216)","(8.98, 3.936594673133431)","(7.33, 4.271161427747611)","(99.0, -1.494883094119821)","(6.17, 1.705936992040895)","(9.08, 1.4947934950523956)",...,"(6.46, 3.262174732382019)","(99.0, 3.8543089836388043)","(99.0, 0.4044176831080435)","(99.0, -0.6528782364059577)","(7.04, 3.0516776797894414)","(7.28, 3.309449594839105)","(99.0, -0.892232913233916)","(7.28, 3.3630601523342953)","(8.25, 3.40953787739230